In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('Downloads/allMLdata/melb_data.csv')

y=data.Price
melb_predictors = data.drop(['Price'], axis=1)
X=melb_predictors.select_dtypes(exclude=['object'])

X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size=0.8, test_size=0.2,random_state=0)
                            

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train,X_valid,y_train,y_valid):
    model = RandomForestRegressor(n_estimators = 10, random_state=0)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [8]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from approach 1 (drop columns):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train,y_valid))

MAE from approach 1 (drop columns):
183550.22137772635


In [14]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.fit_transform(X_valid))
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


print("MAE from approach 2 (imputation):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train,y_valid))

MAE from approach 2 (imputation):
183550.22137772635


In [16]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

for col in cols_with_missing:
    X_train_plus[col+'_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col+'_was_missing'] = X_valid_plus[col].isnull()

    
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.fit_transform(X_valid_plus))
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


print("MAE from approach 3 (extended imputation):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train,y_valid))

MAE from approach 3 (extended imputation):
183550.22137772635


In [17]:
print(X_train.shape)

missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
